<a href="https://colab.research.google.com/github/khushbooekhande/DrugRecommendationSystem/blob/master/DiseasePrediction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import numpy as np
import pandas as pd
import csv
from collections import defaultdict

In [97]:
from google.colab import drive
drive.mount("/content/drive")

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [0]:
data = pd.read_excel('/content/drive/My Drive/gtdt/Symptoms_dataset.xlsx')

In [0]:
data = data.fillna(method='ffill')

In [0]:
# Clean Disease and Symptom Names
def clean_data(data):
    data_list = []
    data_name = data.replace('^','_').split('_')
    n = 1
    for names in data_name:
        if (n % 2 == 0):
            data_list.append(names)
        n += 1
    return data_list

In [0]:
disease_list = []
disease_symptom_dict = defaultdict(list)
disease_symptom_count = {}
count = 0

for idx, row in data.iterrows():
    
    # Get the Disease Names
    if (row['Disease'] !="\xc2\xa0") and (row['Disease'] != ""):
        disease = row['Disease']
        disease_list = clean_data(data=disease)
        count = row['Count of Disease Occurrence']

    # Get the Symptoms Corresponding to Diseases
    if (row['Symptom'] !="\xc2\xa0") and (row['Symptom'] != ""):
        symptom = row['Symptom']
        symptom_list = clean_data(data=symptom)
        for d in disease_list:
            for s in symptom_list:
                disease_symptom_dict[d].append(s)
            disease_symptom_count[d] = count

In [0]:
# Writing the data in a csv file

with open("/content/drive/My Drive/gtdt/Symptoms_clean.csv","w") as csvfile:
    writer = csv.writer(csvfile)
    for key,values in disease_symptom_dict.items():
        for v in values:
            #key = str.encode(key)
            key = str.encode(key).decode('utf-8')
            #.strip()
            #v = v.encode('utf-8').strip()
            #v = str.encode(v)
            writer.writerow([key,v,disease_symptom_count[key]])

In [0]:
columns = ['Disease', 'Symptoms', 'Count']
data_clean = pd.read_csv("/content/drive/My Drive/gtdt/Symptoms_clean.csv", names = columns, encoding ="ISO-8859-1")

In [104]:
data_clean.head()

,Disease,Symptoms,Count
0,hypertensive disease,pain chest,3363.0
1,hypertensive disease,shortness of breath,3363.0
2,hypertensive disease,dizziness,3363.0
3,hypertensive disease,asthenia,3363.0
4,hypertensive disease,fall,3363.0


In [0]:
data_clean.to_csv("/content/drive/My Drive/gtdt/Symptoms_clean.csv", index=False)

In [0]:
slist = []
dlist = []
with open("/content/drive/My Drive/gtdt/nodetable.csv","w") as csvfile:
    writer = csv.writer(csvfile)

    for key,values in disease_symptom_dict.items():
        for v in values:
            if v not in slist:
                writer.writerow([v,v,"symptom"])
                slist.append(v)
        if key not in dlist:
            writer.writerow([key,key,"disease"])
            dlist.append(key)

In [0]:
nt_columns = ['Id','Label','Attribute']

In [0]:
nt_data = pd.read_csv("/content/drive/My Drive/gtdt/nodetable.csv",names=nt_columns, encoding ="ISO-8859-1",)

In [109]:
nt_data.head()

,Id,Label,Attribute
0,pain chest,pain chest,symptom
1,shortness of breath,shortness of breath,symptom
2,dizziness,dizziness,symptom
3,asthenia,asthenia,symptom
4,fall,fall,symptom


In [0]:
nt_data.to_csv("/content/drive/My Drive/gtdt/nodetable.csv", index = False)

In [0]:
data_clean= pd.read_csv("/content/drive/My Drive/gtdt/Symptoms_clean.csv", encoding ="ISO-8859-1",)

In [112]:
data_clean.head()

,Disease,Symptoms,Count
0,hypertensive disease,pain chest,3363.0
1,hypertensive disease,shortness of breath,3363.0
2,hypertensive disease,dizziness,3363.0
3,hypertensive disease,asthenia,3363.0
4,hypertensive disease,fall,3363.0


In [113]:
len(data_clean["Disease"].unique())

149

In [114]:
len(data_clean["Symptoms"].unique())

405

In [0]:
df = pd.DataFrame(data_clean)

In [0]:
df_1 = pd.get_dummies(df.Symptoms)

In [117]:
df_1.head()

,Heberden's node,Murphy's sign,Stahli's line,abdomen acute,abdominal bloating,abdominal tenderness,abnormal sensation,abnormally hard consistency,abortion,abscess bacterial,absences finding,achalasia,ache,adverse effect,adverse reaction,agitation,air fluid level,alcohol binge episode,alcoholic withdrawal symptoms,ambidexterity,angina pectoris,anorexia,anosmia,aphagia,apyrexial,arthralgia,ascites,asterixis,asthenia,asymptomatic,ataxia,atypia,aura,awakening early,barking cough,bedridden,behavior hyperactive,behavior showing increased motor activity,blackout,blanch,...,tenesmus,terrify,thicken,throat sore,throbbing sensation quality,tinnitus,tired,titubation,todd paralysis,tonic seizures,transaminitis,transsexual,tremor,tremor resting,tumor cell invasion,unable to concentrate,unconscious state,uncoordination,underweight,unhappy,unresponsiveness,unsteady gait,unwell,urge incontinence,urgency ofÃÂ micturition,urinary hesitation,urinoma,verbal auditory hallucinations,verbally abusive behavior,vertigo,vision blurred,vomiting,weepiness,weight gain,welt,wheelchair bound,wheezing,withdraw,worry,yellow sputum
0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [118]:
df.head()

,Disease,Symptoms,Count
0,hypertensive disease,pain chest,3363.0
1,hypertensive disease,shortness of breath,3363.0
2,hypertensive disease,dizziness,3363.0
3,hypertensive disease,asthenia,3363.0
4,hypertensive disease,fall,3363.0


In [0]:
df_s = df['Disease']

In [0]:
df_pivoted = pd.concat([df_s,df_1], axis=1)

In [0]:
df_pivoted.drop_duplicates(keep='first',inplace=True)

In [122]:
df_pivoted[:5]

,Disease,Heberden's node,Murphy's sign,Stahli's line,abdomen acute,abdominal bloating,abdominal tenderness,abnormal sensation,abnormally hard consistency,abortion,abscess bacterial,absences finding,achalasia,ache,adverse effect,adverse reaction,agitation,air fluid level,alcohol binge episode,alcoholic withdrawal symptoms,ambidexterity,angina pectoris,anorexia,anosmia,aphagia,apyrexial,arthralgia,ascites,asterixis,asthenia,asymptomatic,ataxia,atypia,aura,awakening early,barking cough,bedridden,behavior hyperactive,behavior showing increased motor activity,blackout,...,tenesmus,terrify,thicken,throat sore,throbbing sensation quality,tinnitus,tired,titubation,todd paralysis,tonic seizures,transaminitis,transsexual,tremor,tremor resting,tumor cell invasion,unable to concentrate,unconscious state,uncoordination,underweight,unhappy,unresponsiveness,unsteady gait,unwell,urge incontinence,urgency ofÃÂ micturition,urinary hesitation,urinoma,verbal auditory hallucinations,verbally abusive behavior,vertigo,vision blurred,vomiting,weepiness,weight gain,welt,wheelchair bound,wheezing,withdraw,worry,yellow sputum
0,hypertensive disease,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,hypertensive disease,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,hypertensive disease,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,hypertensive disease,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,hypertensive disease,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [123]:
len(df_pivoted)

2116

In [0]:
cols = df_pivoted.columns

In [0]:
cols = cols[1:]

In [126]:
df_pivoted = df_pivoted.groupby('Disease').sum()
df_pivoted = df_pivoted.reset_index()
df_pivoted[:5]

,Disease,Heberden's node,Murphy's sign,Stahli's line,abdomen acute,abdominal bloating,abdominal tenderness,abnormal sensation,abnormally hard consistency,abortion,abscess bacterial,absences finding,achalasia,ache,adverse effect,adverse reaction,agitation,air fluid level,alcohol binge episode,alcoholic withdrawal symptoms,ambidexterity,angina pectoris,anorexia,anosmia,aphagia,apyrexial,arthralgia,ascites,asterixis,asthenia,asymptomatic,ataxia,atypia,aura,awakening early,barking cough,bedridden,behavior hyperactive,behavior showing increased motor activity,blackout,...,tenesmus,terrify,thicken,throat sore,throbbing sensation quality,tinnitus,tired,titubation,todd paralysis,tonic seizures,transaminitis,transsexual,tremor,tremor resting,tumor cell invasion,unable to concentrate,unconscious state,uncoordination,underweight,unhappy,unresponsiveness,unsteady gait,unwell,urge incontinence,urgency ofÃÂ micturition,urinary hesitation,urinoma,verbal auditory hallucinations,verbally abusive behavior,vertigo,vision blurred,vomiting,weepiness,weight gain,welt,wheelchair bound,wheezing,withdraw,worry,yellow sputum
0,Alzheimer's disease,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0
1,HIV,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,PneumocystisÃÂ cariniiÃÂ pneumonia,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1
3,accidentÃÂ cerebrovascular,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,acquiredÃÂ immuno-deficiency syndrome,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [127]:
len(df_pivoted)

149

In [0]:
df_pivoted.to_csv("/content/drive/My Drive/gtdt/Symptoms_encoded.csv")

In [0]:
x = df_pivoted[cols]
y = df_pivoted['Disease']

In [130]:
x

,Heberden's node,Murphy's sign,Stahli's line,abdomen acute,abdominal bloating,abdominal tenderness,abnormal sensation,abnormally hard consistency,abortion,abscess bacterial,absences finding,achalasia,ache,adverse effect,adverse reaction,agitation,air fluid level,alcohol binge episode,alcoholic withdrawal symptoms,ambidexterity,angina pectoris,anorexia,anosmia,aphagia,apyrexial,arthralgia,ascites,asterixis,asthenia,asymptomatic,ataxia,atypia,aura,awakening early,barking cough,bedridden,behavior hyperactive,behavior showing increased motor activity,blackout,blanch,...,tenesmus,terrify,thicken,throat sore,throbbing sensation quality,tinnitus,tired,titubation,todd paralysis,tonic seizures,transaminitis,transsexual,tremor,tremor resting,tumor cell invasion,unable to concentrate,unconscious state,uncoordination,underweight,unhappy,unresponsiveness,unsteady gait,unwell,urge incontinence,urgency ofÃÂ micturition,urinary hesitation,urinoma,verbal auditory hallucinations,verbally abusive behavior,vertigo,vision blurred,vomiting,weepiness,weight gain,welt,wheelchair bound,wheezing,withdraw,worry,yellow sputum
0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1
3,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
144,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,1,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
145,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,0,0,0,0,0,0,0,0,0
146,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0
147,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0


Prediction model


In [0]:
from sklearn.model_selection import train_test_split
from sklearn.ensemble import ExtraTreesClassifier
from sklearn.naive_bayes import MultinomialNB
from sklearn.tree import DecisionTreeClassifier
from sklearn import svm
from sklearn.naive_bayes import GaussianNB
from sklearn.ensemble import RandomForestClassifier

Multinomial Naive Bayes

In [0]:
mnb = MultinomialNB()
mnb = mnb.fit(x, y)
y_pred = mnb.predict(x)

In [133]:
pred_counts = 0
for i in range(0, len( y.values)):
  if y_pred[i]!= y.values[i]:
    pred_counts += 1
    print('Actual Disease:', y.values[i])
    print('Predicted Disease using Model:',y_pred[i])
    print('\n')
print("Total number of incorrect predictions:", pred_counts)
print("Accuracy of the MNB model in Predicting Disease: ", ((len(y)-pred_counts)/len(y))*100, '%')

Actual Disease: acquiredÃÂ immuno-deficiency syndrome
Predicted Disease using Model: HIV


Actual Disease: cholelithiasis
Predicted Disease using Model: biliary calculus


Actual Disease: coronary heart disease
Predicted Disease using Model: coronary arteriosclerosis


Actual Disease: depressive disorder
Predicted Disease using Model: depression mental


Actual Disease: hiv infections
Predicted Disease using Model: HIV


Actual Disease: malignant neoplasm of breast
Predicted Disease using Model: carcinoma breast


Actual Disease: malignant neoplasm of lung
Predicted Disease using Model: carcinoma of lung


Actual Disease: malignant neoplasm of prostate
Predicted Disease using Model: carcinoma prostate


Actual Disease: malignant tumor of colon
Predicted Disease using Model: carcinoma colon


Actual Disease: oralcandidiasis
Predicted Disease using Model: candidiasis


Actual Disease: pericardial effusion body substance
Predicted Disease using Model: effusion pericardial


Actual Diseas

ExtraTree Classifier 


In [0]:
ext = ExtraTreesClassifier(n_estimators=100, random_state=0)
ext = ext.fit(x, y)
y_pred = ext.predict(x)

In [135]:
pred_counts = 0
for i in range(0, len( y.values)):
  if y_pred[i]!= y.values[i]:
    pred_counts += 1
    print('Actual Disease:', y.values[i])
    print('Predicted Disease using Model:',y_pred[i])
    print('\n')
print("Total number of incorrect predictions:", pred_counts)
print("Accuracy of the ExtraTree classifier model in Predicting Disease: ", ((len(y)-pred_counts)/len(y))*100, '%')

Actual Disease: acquiredÃÂ immuno-deficiency syndrome
Predicted Disease using Model: HIV


Actual Disease: cholelithiasis
Predicted Disease using Model: biliary calculus


Actual Disease: coronary heart disease
Predicted Disease using Model: coronary arteriosclerosis


Actual Disease: depressive disorder
Predicted Disease using Model: depression mental


Actual Disease: hiv infections
Predicted Disease using Model: HIV


Actual Disease: malignant neoplasm of breast
Predicted Disease using Model: carcinoma breast


Actual Disease: malignant neoplasm of lung
Predicted Disease using Model: carcinoma of lung


Actual Disease: malignant neoplasm of prostate
Predicted Disease using Model: carcinoma prostate


Actual Disease: malignant tumor of colon
Predicted Disease using Model: carcinoma colon


Actual Disease: oralcandidiasis
Predicted Disease using Model: candidiasis


Actual Disease: pericardial effusion body substance
Predicted Disease using Model: effusion pericardial


Actual Diseas

Decision Tree classifier

In [0]:
for i in range(10, 101, 20):
  dtc = DecisionTreeClassifier(max_depth=i)
  dtc_1 = dtc.fit(x,y)
  y_pred = dtc_1.predict(x)
  pred_counts = 0
  for i in range(0, len( y.values)):
    if y_pred[i]!= y.values[i]:
      pred_counts += 1
      print('Actual Disease:', y.values[i])
      print('Predicted Disease using Model:',y_pred[i])
      print('\n')
  print("Total number of incorrect predictions:", pred_counts)
  print("Accuracy of the DecisionTree classifier model in Predicting Disease: ", ((len(y)-pred_counts)/len(y))*100, '%')
  print("\n")
  

Taking symptom as input and predicting disease.

In [0]:
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt

importances = dtc.feature_importances_
indices = np.argsort(importances)[::-1]

features = cols

In [139]:
#Getting the top 10 symptoms.
for f in range(10):
    print("%d. feature %d - %s (%f)" % (f + 1, indices[f], features[indices[f]] ,importances[indices[f]]))

1. feature 332 - sleepy (0.022556)
2. feature 356 - sweating increased (0.022556)
3. feature 363 - tachypnea (0.015038)
4. feature 235 - nausea (0.015038)
5. feature 97 - dyspnea on exertion (0.015038)
6. feature 38 - blackout (0.015038)
7. feature 187 - hyponatremia (0.015038)
8. feature 323 - scratch marks (0.015038)
9. feature 296 - prostatism (0.015038)
10. feature 254 - pain (0.015038)


In [0]:
feature_dict = {}
for i,f in enumerate(features):
    feature_dict[f] = i

In [141]:
feature_dict['sleepy']

332

In [0]:
symptomDR = [i/332 if i == 332 else i*0 for i in range(len(features))]

In [143]:
len(symptomDR)

404

In [0]:
symptomDR = np.array(symptomDR).reshape(1,len(symptomDR))

In [145]:
dtc.predict(symptomDR)

array(['failure heart congestive'], dtype=object)

In [146]:
dtc.predict_proba(symptomDR)

array([[0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0.]])

Support Vector Machine Classifier

In [0]:
svm = svm.SVC()
svm = svm.fit(x, y)
y_pred = svm.predict(x)

In [148]:
pred_counts = 0
for i in range(0, len( y.values)):
  if y_pred[i]!= y.values[i]:
    pred_counts += 1
    print('Actual Disease:', y.values[i])
    print('Predicted Disease using Model:',y_pred[i])
    print('\n')
print("Total number of incorrect predictions:", pred_counts)
print("Accuracy of the SVM model in Predicting Disease: ", ((len(y)-pred_counts)/len(y))*100, '%')

Actual Disease: HIV
Predicted Disease using Model: hiv infections


Actual Disease: acquiredÃÂ immuno-deficiency syndrome
Predicted Disease using Model: hiv infections


Actual Disease: biliary calculus
Predicted Disease using Model: cholelithiasis


Actual Disease: candidiasis
Predicted Disease using Model: oralcandidiasis


Actual Disease: carcinoma breast
Predicted Disease using Model: malignant neoplasm of breast


Actual Disease: carcinoma colon
Predicted Disease using Model: malignant tumor of colon


Actual Disease: carcinoma of lung
Predicted Disease using Model: malignant neoplasm of lung


Actual Disease: carcinoma prostate
Predicted Disease using Model: malignant neoplasm of prostate


Actual Disease: coronary arteriosclerosis
Predicted Disease using Model: coronary heart disease


Actual Disease: depression mental
Predicted Disease using Model: depressive disorder


Actual Disease: effusion pericardial
Predicted Disease using Model: pericardial effusion body substance


Ac

Naive Bayes Classifier

In [0]:
naivebys = GaussianNB()
naivebys = naivebys.fit(x, y)
y_pred = naivebys.predict(x)

In [150]:
pred_counts = 0
for i in range(0, len( y.values)):
  if y_pred[i]!= y.values[i]:
    pred_counts += 1
    print('Actual Disease:', y.values[i])
    print('Predicted Disease using Model:',y_pred[i])
    print('\n')
print("Total number of incorrect predictions:", pred_counts)
print("Accuracy of the GNB model in Predicting Disease: ", ((len(y)-pred_counts)/len(y))*100, '%')

Actual Disease: acquiredÃÂ immuno-deficiency syndrome
Predicted Disease using Model: HIV


Actual Disease: cholelithiasis
Predicted Disease using Model: biliary calculus


Actual Disease: coronary heart disease
Predicted Disease using Model: coronary arteriosclerosis


Actual Disease: depressive disorder
Predicted Disease using Model: depression mental


Actual Disease: hiv infections
Predicted Disease using Model: HIV


Actual Disease: malignant neoplasm of breast
Predicted Disease using Model: carcinoma breast


Actual Disease: malignant neoplasm of lung
Predicted Disease using Model: carcinoma of lung


Actual Disease: malignant neoplasm of prostate
Predicted Disease using Model: carcinoma prostate


Actual Disease: malignant tumor of colon
Predicted Disease using Model: carcinoma colon


Actual Disease: oralcandidiasis
Predicted Disease using Model: candidiasis


Actual Disease: pericardial effusion body substance
Predicted Disease using Model: effusion pericardial


Actual Diseas

Random Forest classifier

In [151]:
for i in range(1, 10, 2):
  rfc = RandomForestClassifier(max_depth=i)
  rfc = rfc.fit(x, y)
  y_pred = rfc.predict(x)
  pred_counts = 0
  for i in range(0, len( y.values)):
    if y_pred[i]!= y.values[i]:
      pred_counts += 1
      print('Actual Disease:', y.values[i])
      print('Predicted Disease using Model:',y_pred[i])
      print('\n')
  print("Total number of incorrect predictions:", pred_counts)
  print("Accuracy of the RF model in Predicting Disease: ", ((len(y)-pred_counts)/len(y))*100, '%')
  print("\n")

Actual Disease: Alzheimer's disease
Predicted Disease using Model: malignant neoplasm of lung


Actual Disease: accidentÃÂ cerebrovascular
Predicted Disease using Model: hemiparesis


Actual Disease: acquiredÃÂ immuno-deficiency syndrome
Predicted Disease using Model: HIV


Actual Disease: affect labile
Predicted Disease using Model: manic disorder


Actual Disease: anemia
Predicted Disease using Model: sickle cell anemia


Actual Disease: arthritis
Predicted Disease using Model: malignant neoplasm of lung


Actual Disease: bacteremia
Predicted Disease using Model: upper respiratory infection


Actual Disease: benign prostatic hypertrophy
Predicted Disease using Model: malignant neoplasm of lung


Actual Disease: biliary calculus
Predicted Disease using Model: malignant neoplasm of lung


Actual Disease: bronchitis
Predicted Disease using Model: diabetes


Actual Disease: candidiasis
Predicted Disease using Model: herniaÃÂ hiatal


Actual Disease: carcinoma
Predicted Disease using M